In [ ]:
!pip install pingouin

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
from pathlib import Path
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import seaborn as sns
import pandas as pd
import scipy.stats as sstats
import pingouin
%matplotlib inline

In [ ]:
plt.rcParams['font.size'] = 7
plt.rcParams["axes.unicode_minus"] = False

In [ ]:
# this is for Haley
data_dir = Path("/content/gdrive/MyDrive/Manuscript_JAMAPsychiatry_EP_MEND_PLS/Data")
figs_dir = Path("/content/gdrive/MyDrive/Manuscript_JAMAPsychiatry_EP_MEND_PLS/Figures")

In [ ]:
 classified_tracts = pd.read_csv(data_dir / "classified_tracts_w_coefs.csv")

In [ ]:
import colorsys

def scale_lightness(rgb, scale_l):
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    return colorsys.hls_to_rgb(h, min(1, l * scale_l), s = s)

In [ ]:
classified_tracts.sort_values(by='MEND_coefs', key=abs, ascending=False)

In [ ]:
classified_tracts[classified_tracts["Tracts"].str.contains("lemn")]

In [ ]:
from matplotlib.colors import ListedColormap, to_rgb, to_hex
tract_type2color = {
    "Tracts in the brainstem": "#471C6C",
    "Commissural tracts": "#367D8F",
    "Association tracts": "#96D98C",
    "Projection tracts": "#F1E400",
}
tract_names = list(tract_type2color.keys())
tract_colors = list(tract_type2color.values())
cmap_tracts = ListedColormap([to_rgb(_) for _ in tract_colors])

In [ ]:
cmap_tracts

In [ ]:
diff_pred_names = np.loadtxt(data_dir / "JHU_labels.txt", delimiter="\t", dtype=str)
# diff_pred_colors = [xload_names.index(_.split("_")[0]) for _ in diff_pred_names]

diff_tract_colors = [tract_names.index(classified_tracts.loc[i, "Groups"]) for i, _ in enumerate(diff_pred_names)]

In [ ]:
classified_tracts["combined_ci"] = (classified_tracts["EP_CI"] > 0).astype(int) + (classified_tracts["MEND_CI"] > 0).astype(int)

In [ ]:
score_corr_r, score_corr_p = sstats.pearsonr(classified_tracts['MEND_coefs'], classified_tracts['EP_coefs'])


fig, ax = plt.subplots(figsize=(4, 4))
scatter = ax.scatter(classified_tracts['MEND_coefs'], classified_tracts['EP_coefs'],
           c=diff_tract_colors, s=8 + classified_tracts["combined_ci"] * 7, cmap=cmap_tracts, linewidths=0.5, clip_on=False)# , edgecolors='white')

# for i in range(len(classified_tracts)):
#     ax.text(classified_tracts.loc[i, 'MEND_coefs'], classified_tracts.loc[i, 'EP_coefs'],
#             "".join(classified_tracts.loc[i, "combined_ci"]*["*"]),
#             fontsize=5, ha="center", va="center")

ax.axvline(x=0, c="k", linewidth=0.5)#, zorder=0.5, ls=(0, (2, 1)))
ax.axhline(y=0, c="k", linewidth=0.5)#, zorder=0.7, ls=(0, (2, 1)))

# ax.fill_between([0, 0.45], [0.4, 0.4], [0, 0], color=to_hex(scale_lightness(to_rgb(xload_colors[1]), 0.7)), zorder=0.5, alpha=0.3)
# ax.fill_between([-0.4, 0], [0, 0], [-0.4, -0.4], color=to_hex(scale_lightness(to_rgb(xload_colors[0]), 0.7)), zorder=0.5, alpha=0.3)

# reg_x = np.arange(-20, np.max(ep_pls_result["results"]["x_scores"][:, 0]), 0.1)
#ax.plot(reg_x, m * reg_x + b, color="gray", zorder=0.5)

ax.set(xlim=(-0.517, 0.517), ylim=(-0.5, 0.5), xticks=[-0.4, -0.2, 0, 0.2, 0.4], yticks=[-0.4, -0.2, 0, 0.2, 0.4],
       xlabel="MEND microstructural loadings", ylabel="HCPEP microstructural loadings")

ax.text(0.1, -0.25, f"r={score_corr_r:.2f}, p={score_corr_p:.2e}")

sns.despine(top=True, right=True, trim=False, ax=ax)
ax.tick_params(axis=u'both', which=u'both', length=1)

handles, previous_labels = scatter.legend_elements()
legend = ax.legend(handles=handles, labels=tract_names, loc="upper left", bbox_to_anchor=(0, 1.0), frameon=False)
ax.add_artist(legend)

fig.savefig(figs_dir / "mend_hcpep_xload_corr_by_tracts.svg")

In [ ]:
sns.kdeplot(data=classified_tracts, x="MEND_coefs", hue="Groups")

In [ ]:
sns.kdeplot(data=classified_tracts, x="EP_coefs", hue="Groups")

In [ ]:
sns.ecdfplot(data=classified_tracts, x="MEND_coefs", hue="Groups")

In [ ]:
sns.ecdfplot(data=classified_tracts, x="EP_coefs", hue="Groups")

In [ ]:
sns.histplot(data=classified_tracts, x="MEND_coefs", hue="Groups", element="poly")

In [ ]:
sns.histplot(data=classified_tracts, x="EP_coefs", hue="Groups", element="poly")

In [ ]:
sns.kdeplot(data=classified_tracts, x="MEND_coefs", y="EP_coefs", hue="Groups")

In [ ]:
sns.kdeplot(data=classified_tracts, x="MEND_coefs", y="EP_coefs", hue="Groups", fill=True, alpha=0.5)